In [ ]:
%%capture
%load_ext rpy2.ipython
%matplotlib inline
from copy import copy
from IPython.display import HTML
from matplotlib import pyplot
import os, numpy as np
np.random.seed(0)
import pandas, statsmodels.api as sm
from selection.algorithms.lasso import lasso, data_carving, additive_noise
from scipy.stats import norm as ndist
from selection.algorithms.lasso import (standard_lasso, instance)
from selection.algorithms.forward_step import forward_step
from selection.distributions.discrete_family import discrete_family
import itable
HTML("""
<style type="text/css">
    .emphred {color: #cc2222; font-size: 100%; font-weight: bold; font-family: arial,helvetica,sans-serif}
    .emphblue {color: #2222cc; font-size: 100%; font-weight: bold; font-family: arial,helvetica,sans-serif}
</style>
""")

# Run with commit  8d7f7d9c2742e450ba95174d1aa8ac5054390d45
# from http://github.com/jonathan-taylor/selective-inference

# Data snooping, data splitting and selective inference


## Outline 

* We saw that using the data to choose the hypothesis to test does not work well.
    
* Running example: model selection with the LASSO  ([arxiv.org/1311.6238](http://arxiv.org/abs/1311.6238v4) )

* Data splitting and variations ([arxiv.org/1410.2597](http://arxiv.org/abs/1410.2597) ).
        
       

# Running example

- In vitro measurement of resistance of sample of HIV viruses to NRTI drug 3TC.

- 633 cases, and 91 different mutations occuring more than 10 times in sample.

- Source: [HIVDB](http://hivdb.stanford.edu/pages/published_analysis/genophenoPNAS2006/)

- Goal: to build an <font class="emphblue">interpretable predictive model</font> of resistance based on mutation pattern.

- Model: $y|X$ (fixed design, Gaussian errors)




In [ ]:
if not os.path.exists("NRTI_DATA.txt"):
    NRTI = pandas.read_table("http://hivdb.stanford.edu/pages/published_analysis/genophenoPNAS2006/DATA/NRTI_DATA.txt", na_values="NA")
else:
    NRTI = pandas.read_table("NRTI_DATA.txt")
NRTI_specific = []
NRTI_muts = []
mixtures = np.zeros(NRTI.shape[0])
for i in range(1,241):
    d = NRTI['P%d' % i]
    for mut in np.unique(d):
        if mut not in ['-','.'] and len(mut) == 1:
            test = np.equal(d, mut)
            if test.sum() > 10:
                NRTI_specific.append(np.array(np.equal(d, mut))) 
                NRTI_muts.append("P%d%s" % (i,mut))

NRTI_specific = NRTI.from_records(np.array(NRTI_specific).T, columns=NRTI_muts)

In [ ]:
# Next, standardize the data, keeping only those where Y is not missing

X_NRTI = np.array(NRTI_specific, np.float)
Y = NRTI['3TC'] # shorthand
keep = ~np.isnan(Y).astype(np.bool)
X_NRTI = X_NRTI[np.nonzero(keep)]; Y=Y[keep]
Y = np.array(np.log(Y), np.float); Y -= Y.mean()
X_NRTI -= X_NRTI.mean(0)[None, :]; X_NRTI /= X_NRTI.std(0)[None,:]
X = X_NRTI # shorthand
np.savetxt("X_3TC.csv", X, delimiter=',')
np.savetxt("Y_3TC.csv", Y, delimiter=',')
file("muts_3TC.csv", 'w').write('\n'.join(NRTI_muts))

In [ ]:
# Design matrix
# Columns are site / amino acid pairs
X.shape

In [ ]:
# Variable names
NRTI_muts[:10], len(NRTI_muts)

In [ ]:
%%capture
# Fit OLS model and plot coefficients

n, p = X.shape
ols_fit = sm.OLS(Y, X).fit()
sigma_3TC = np.linalg.norm(ols_fit.resid) / np.sqrt(n-p-1)
OLS_3TC = ols_fit.params

fig_3TC = pyplot.figure(figsize=(24,12))
ax_3TC = fig_3TC.gca()
ax_3TC.bar(np.arange(1, len(NRTI_muts)+1)-0.5, OLS_3TC, label='OLS', color='red', alpha=0.7)                                                          
ax_3TC.set_xticks(range(1, (len(NRTI_muts)+1)))
ax_3TC.set_xticklabels(NRTI_muts, rotation='vertical', fontsize=18) 
ax_3TC.set_xlim([-1, len(NRTI_muts)+1])
ax_3TC.set_title(r'OLS coefficients for 3TC resistance, $\hat{\sigma}$=%0.1e' % sigma_3TC, fontsize=50)
ax_3TC.set_ylabel('Parameter estimates', fontsize=30)
ax_3TC.legend(fontsize=30)
;

In [ ]:
fig_3TC

## LASSO 

- Another popular penalized regression technique.
- Use the standardized model.
- The LASSO estimate is
$$
\hat{\beta}_{\lambda} = \text{argmin}_{\beta} \frac{1}{2} \|Y-X\beta\|^2_2 + \lambda \|\beta\|_1$$
where
$$
\|\beta\|_1 = \sum_{j=1}^p |\beta_j|
$$
is the $\ell_1$ norm.

- Corresponds (through Lagrange multiplier) to an $\ell^1$ constraint on ${\beta_{}}$’s. 

- In theory and practice, it works well when many ${\beta_{j}}$’s are 0 and gives "sparse" solutions.

- It is a (computable) approximation to the best subsets AIC model.

- It is computable because the minimization problem is a convex problem.

## Why do we get sparse solutions with the LASSO?

<img src="http://stats203.stanford.edu/notebooks/figs/lassofig.png">

In [ ]:
%%R -i X,Y
library(lars)
plot(lars(X, as.numeric(Y), type='lasso'))

In [ ]:
%%R
data(diabetes)
plot(lars(diabetes$x, diabetes$y))

# Model selection with the LASSO 


#### Choice of $\lambda$ 

$$
\lambda = \kappa \cdot \mathbb{E}( \max_j\|X^T_j\epsilon\|), \qquad \epsilon \sim N(0, \sigma^2 I).
$$


#### $\sigma^2$ the usual estimate from full model.

#### Used $\kappa=1$


#### [arxiv.org/1311.6238](http://arxiv.org/abs/1311.6238)




In [ ]:
# Choose a LASSO parameter
lambda_theoretical = []
for i in range(10000):
    lambda_theoretical.append(np.fabs(np.dot(X.T, np.random.standard_normal(n))).max())
lambda_theoretical = np.round((1 * np.mean(lambda_theoretical) * sigma_3TC))


### Variables chosen for 3TC

In [ ]:
lambda_theoretical

In [ ]:
from selection.algorithms.lasso import lasso
lasso_3TC = lasso(Y, X, lambda_theoretical, sigma=sigma_3TC)
lasso_3TC.fit()
active_3TC = [NRTI_muts[i] for i in lasso_3TC.active]

In [ ]:
active_3TC

# Inference after LASSO?

### Can we use usual inference after running these algorithms?

## <font class="emphred">No! We have been data snooping!</font>


### Should we throw out all these algorithms?

## <font class="emphred">No! These algorithms often return pretty good models!</font>

### We must address inherent conflict between <font class="emphred">exploration</font> and <font class="emphred">inference</font>.


## Exploratory Data Analysis (EDA)

<img src="http://upload.wikimedia.org/wikipedia/en/thumb/e/e9/John_Tukey.jpg/220px-John_Tukey.jpg">

* From [Wikipedia entry on EDA](http://en.wikipedia.org/wiki/Exploratory_data_analysis)

<h4> ... <font class="emphblue">more emphasis on</font> ...
<font class="emphblue">using data to suggest hypotheses to test.</font> 
</h4>

## Exploratory Data Analysis (EDA)

<img src="http://upload.wikimedia.org/wikipedia/en/thumb/e/e9/John_Tukey.jpg/220px-John_Tukey.jpg">


<h4>...  <font class="emphred">bias</font> ... <font class="emphred">testing hypotheses suggested by the data</font>.
</h4>



# Selective inference

#### Allows <font class="emphred">testing hypotheses suggested by the data.</font>

#### <font class="emphred">No free lunch:</font> users must declare the <font class="emphred">algorithm</font>.

#### Simplest example of selective inference: data splitting

# Data splitting: simplest example


1. $\omega \sim \mathbb{Q}$ a random partition of size $(n_1(\omega),n-n_1(\omega))$.

2. Choose $\hat{E} = \hat{E}(y_1(y,\omega))$

3. Report results for model $\hat{E}$ using data $y_2(y,\omega)$.



In [ ]:
%%capture
np.random.seed(0)
active_carve, selective_pval_carve, selective_interval_carve, split_pval, split_interval = [], [], [], [], []
results = data_carving(Y, X, lam_frac=1., sigma=sigma_3TC, burnin=2000, ndraw=8000, split_frac=0.9,
                       splitting=True, compute_intervals=False)[0]
for result in results:
    active_carve.append(NRTI_muts[result[0]])
    selective_pval_carve.append(result[1])
    selective_interval_carve.append(result[2])
    split_pval.append(result[3])
    split_interval.append(result[4])
carve_pvalues = pandas.DataFrame({'Mutation':active_carve, 
                                  'Data splitting':split_pval, 
                                  'Data carving':np.maximum(selective_pval_carve, 0)})
carve_pvalues = carve_pvalues.reindex_axis(['Mutation', 'Data splitting', 'Data carving'], axis=1)
carve_pvalue_table = itable.PrettyTable(carve_pvalues, tstyle=itable.TableStyle(theme="theme1"), center=True)
signif_carve = carve_pvalues['Data carving'] < 0.05
signif_split = carve_pvalues['Data splitting'] < 0.05
carve_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(signif_split)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
carve_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(~signif_split)[0], format_function=lambda p: "%0.3f" % p)
carve_pvalue_table.set_cell_style(cols=[2], rows=np.nonzero(signif_carve)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
carve_pvalue_table.set_cell_style(cols=[2], rows=np.nonzero(~signif_carve)[0], format_function=lambda p: "%0.3f" % p)

split_pvalues = carve_pvalues.reindex_axis(['Mutation', 'Data splitting'], axis=1)
split_pvalue_table = itable.PrettyTable(split_pvalues, tstyle=itable.TableStyle(theme="theme1"), center=True)
signif_split = carve_pvalues['Data splitting'] < 0.05
split_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(signif_split)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
split_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(~signif_split)[0], format_function=lambda p: "%0.3f" % p)

active_carve_old = copy(active_carve)


In [ ]:
# randomly split 90/10 percent
# used 90% to find model
# remaining 10% to test hypotheses
split_pvalue_table 


# Data splitting

### Pro: Easy to implement

### Con:  (Minor?) Different $\omega$ yield different statistics on (possibly) different models

### Con: Suffers from <font class="emphred">$p$-value lottery</font>. Any data used for selection cannot be used for inference...


# Data splitting




### Formal justification

$$
{\cal L}_{F \times \mathbb{Q}}(X_{E,2}^Ty_2) = {\cal L}_F(X_{2,E}^Ty_2 | \hat{E}, \omega, y_1)  
$$



### Model used for inference

$$
F \in \left\{N(X_E\beta_E, \sigma_E^2 I): \beta_E \in \mathbb{R}^E \right\} = {\cal M}_E.
$$

### Note

$$
{\cal L}_{F \times \mathbb{Q}} \left( X_{2,E}^Ty_2 \bigl\vert \hat{E}(y, \omega)=E, \omega, y_1 \right)  \equiv {\cal L}_{F\times \mathbb{Q}} \left(X_{E}^Ty  \bigl\vert \hat{E}(y, \omega)=E, \omega, y_1\right)
$$



In [ ]:
%%capture
np.random.seed(20)
active_carve, selective_pval_carve, selective_interval_carve, split_pval, split_interval = [], [], [], [], []
results = data_carving(Y, X, lam_frac=1., sigma=sigma_3TC, burnin=2000, ndraw=8000, split_frac=0.9,
                       splitting=True, compute_intervals=False)[0]
for result in results:
    active_carve.append(NRTI_muts[result[0]])
    selective_pval_carve.append(result[1])
    selective_interval_carve.append(result[2])
    split_pval.append(result[3])
    split_interval.append(result[4])
carve_pvalues = pandas.DataFrame({'Mutation':active_carve, 
                                  'Data splitting':split_pval, 
                                  'Data carving':np.maximum(selective_pval_carve, 0)})
carve_pvalues = carve_pvalues.reindex_axis(['Mutation', 'Data splitting', 'Data carving'], axis=1)
carve_pvalue_table = itable.PrettyTable(carve_pvalues, tstyle=itable.TableStyle(theme="theme1"), center=True)
signif_carve = carve_pvalues['Data carving'] < 0.05
signif_split = carve_pvalues['Data splitting'] < 0.05
carve_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(signif_split)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
carve_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(~signif_split)[0], format_function=lambda p: "%0.3f" % p)
carve_pvalue_table.set_cell_style(cols=[2], rows=np.nonzero(signif_carve)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
carve_pvalue_table.set_cell_style(cols=[2], rows=np.nonzero(~signif_carve)[0], format_function=lambda p: "%0.3f" % p)

split_pvalues = carve_pvalues.reindex_axis(['Mutation', 'Data splitting'], axis=1)
split_pvalue_table = itable.PrettyTable(split_pvalues, tstyle=itable.TableStyle(theme="theme1"), center=True)
signif_split = carve_pvalues['Data splitting'] < 0.05
split_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(signif_split)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
split_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(~signif_split)[0], format_function=lambda p: "%0.3f" % p)

# what is different from other split

different_muts = [active_carve.index(mutn) for mutn in set(active_carve).difference(active_carve_old)]

split_pvalue_table.set_cell_style(cols=[0,1], rows=different_muts,
                                  color='blue', format_function=lambda p: "%0.3f" % p)




In [ ]:
# here is a different 90/10 random split
# variables chosen are different as are p-values
split_pvalue_table

# Selective inference after LASSO 


## Why not use all of the data to choose a model!

$$
{\cal L}_{F }\left(X^T_Ey  \bigl| \hat{E}(y)\right)
$$



#### [arxiv.org/1311.6238](http://arxiv.org/abs/1311.6238),  [arxiv.org/1410.2597](http://arxiv.org/abs/1410.2597).




## Variables chosen for 3TC

In [ ]:
lambda_theoretical

In [ ]:
from selection.algorithms.lasso import lasso
lasso_3TC = lasso(Y, X, lambda_theoretical, sigma=sigma_3TC)
lasso_3TC.fit()
active_3TC = [NRTI_muts[i] for i in lasso_3TC.active]

In [ ]:
active_3TC

In [ ]:
%%capture
ax_3TC.bar(np.arange(1, len(NRTI_muts)+1)-0.5, lasso_3TC.soln, label='LASSO', color='gray') 
ax_3TC.set_title(r'LASSO coefficients ($\lambda=%0.1f$)' % lambda_theoretical, fontsize=50)
ax_3TC.legend(fontsize=30, loc='upper left')

In [ ]:
fig_3TC

In [ ]:
%%capture
selected_OLS = sm.OLS(Y, X[:,lasso_3TC.active]).fit()
OLS_lower, OLS_upper = np.asarray(selected_OLS.conf_int()).T
selective_lower = lasso_3TC.intervals['lower'] # [i[0] for _, _, _, i in lasso_3TC.intervals]
selective_upper = lasso_3TC.intervals['upper'] # [i[1] for _, _, _, i in lasso_3TC.intervals]

selective_intervals = ([['Mutation', 'OLS Lower', 'OLS Upper', 'Selective Lower', 'Selective Upper']] + 
                       zip(active_3TC, OLS_lower, OLS_upper, selective_lower, selective_upper))


In [ ]:
%%capture
fig_select = pyplot.figure(figsize=(24,12))
ax_select = fig_select.gca()
ax_select.bar(np.arange(1, len(active_3TC)+1)-0.5, selected_OLS.params, color='grey', alpha=0.5)                                                          
ax_select.set_xticks(range(1, (len(active_3TC)+1)))
ax_select.set_xticklabels(active_3TC, rotation='vertical', fontsize=18) 
ax_select.set_xlim([0, len(active_3TC)+1])
ax_select.set_title(r'Intervals' % sigma_3TC, fontsize=50)
ax_select.set_ylabel('Parameter values', fontsize=30)
ax_select.errorbar(np.arange(1, (len(active_3TC)+1)), selected_OLS.params, 
                yerr=[selected_OLS.params - OLS_lower, 
                      OLS_upper - selected_OLS.params],
                capsize=10,
                capthick=5,
                fmt=None,
                label='OLS (naive)',
                elinewidth=3,
                ecolor='k')
ax_select.legend(fontsize=30, loc='upper left', numpoints=1)
ax_select.plot([0, len(active_3TC)+1], [0,0], 'k--')

In [ ]:
fig_select

In [ ]:
%%capture
ax_select.errorbar(np.arange(1, (len(active_3TC)+1)), selected_OLS.params,
                yerr=[selected_OLS.params - selective_lower, 
                      selective_upper - selected_OLS.params],
                capsize=10,
                capthick=5,
                fmt=None,
                label='Selective',
                elinewidth=3,
                ecolor='r')
ax_select.legend(fontsize=30, loc='upper left', numpoints=1)

In [ ]:
fig_select

### Some intervals seem to be long...

# Selective inference  for LASSO

### Inference is based on the distribution

$$
{\cal L}_F\left(X^T_Ey  \big\vert (\hat{E}, z_{\hat{E}}) = (E, z_E) \right)
$$

### Could use (at computational cost)

$$
{\cal L}_F\left(X^T_Ey  \big\vert \hat{E}=E \right)
$$

### Model  (same as data splitting)

$$
F \in \left\{N(X_E\beta_E, \sigma^2_E I): \beta_E \in \mathbb{R}^E \right\} ={\cal M}_E.
$$




# Selective inference  for LASSO

### Selection event

- Event of interest
$$
\left\{(\hat{E}, z_{\hat{E}}) = (E,z_E) \right\}
$$

- KKT conditions give us a description of the event:
$$
\begin{aligned}
X_E^T(y-X_E\hat{\beta}_E(y)) &= \lambda z_E \\
\max_{j \not \in E}|X_{j}^T(y - X_E\hat{\beta}_E(y))|& \leq \lambda.
\end{aligned}
$$
with $\text{sign}(\hat{\beta}_E(y)) = z_E$.

- Equivalent to polyhedral constraints

$$
\left\{y: A_{(E,z_E,\lambda)}y \leq b_{(E,z_E,\lambda)} \right\}
$$




## Visualizing LASSO partition 

<img src="http://statweb.stanford.edu/~jtaylo/talks/bids2015/fig_lasso0.png" width="500">

(Credit [Naftali Harris](http://www.naftaliharris.com/blog/lasso-polytope-geometry/))

## What we know after conditioning

<img src="http://statweb.stanford.edu/~jtaylo/talks/bids2015/fig_lasso3.png" width="500">


In [ ]:
selective_pvalues = pandas.DataFrame({'Mutation':active_3TC, 'Naive OLS':selected_OLS.pvalues, 
                                      'Selective':[p for _, p in lasso_3TC.active_pvalues]})
pvalue_table = itable.PrettyTable(selective_pvalues, tstyle=itable.TableStyle(theme="theme1"), center=True)
signif_select = selective_pvalues['Selective'] < 0.05
signif_OLS = selective_pvalues['Naive OLS'] < 0.05
pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(signif_OLS)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(~signif_OLS)[0], format_function=lambda p: "%0.3f" % p)
pvalue_table.set_cell_style(cols=[2], rows=np.nonzero(signif_select)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
pvalue_table.set_cell_style(cols=[2], rows=np.nonzero(~signif_select)[0], format_function=lambda p: "%0.3f" % p)


In [ ]:
pvalue_table

# Data carving


### Use only some data for model selection, all for inference


$$
{\cal L}_{F \times \mathbb{Q}}\left(X^T_Ey \ \bigl \vert \ \hat{E}(y,\omega)=E, \omega \right)
$$

### Recall data splitting

$$
{\cal L}_{F \times \mathbb{Q}}\left(X^T_Ey \ \bigl \vert \ \hat{E}(y,\omega)=E, y_1, \omega\right)
$$

### Improves $p$-value lottery effect

####  [arxiv.org/1410.2597](http://arxiv.org/abs/1410.2597)


In [ ]:
%%capture
np.random.seed(0)
active_carve, selective_pval_carve, selective_interval_carve, split_pval, split_interval = [], [], [], [], []
results = data_carving(Y, X, lam_frac=1., sigma=sigma_3TC, burnin=2000, ndraw=8000, split_frac=0.9,
                       splitting=True, compute_intervals=False)[0]
for result in results:
    active_carve.append(NRTI_muts[result[0]])
    selective_pval_carve.append(result[1])
    selective_interval_carve.append(result[2])
    split_pval.append(result[3])
    split_interval.append(result[4])
carve_pvalues = pandas.DataFrame({'Mutation':active_carve, 
                                  'Data splitting':split_pval, 
                                  'Data carving':np.maximum(selective_pval_carve, 0)})
carve_pvalues = carve_pvalues.reindex_axis(['Mutation', 'Data splitting', 'Data carving'], axis=1)
carve_pvalue_table = itable.PrettyTable(carve_pvalues, tstyle=itable.TableStyle(theme="theme1"), center=True)
signif_carve = carve_pvalues['Data carving'] < 0.05
signif_split = carve_pvalues['Data splitting'] < 0.05
carve_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(signif_split)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
carve_pvalue_table.set_cell_style(cols=[1], rows=np.nonzero(~signif_split)[0], format_function=lambda p: "%0.3f" % p)
carve_pvalue_table.set_cell_style(cols=[2], rows=np.nonzero(signif_carve)[0], color='red', font_weight='bold', format_function=lambda p: "%0.3f" % p)
carve_pvalue_table.set_cell_style(cols=[2], rows=np.nonzero(~signif_carve)[0], format_function=lambda p: "%0.3f" % p)


In [ ]:
carve_pvalue_table

# Data splitting

<img src="http://statweb.stanford.edu/~jtaylo/talks/bids2015/ROC_1.png" width="500">




# Data carving

<img src="http://statweb.stanford.edu/~jtaylo/talks/bids2015/ROC_2.png" width="500">


#### [arxiv.org/1410.2597](http://arxiv.org/abs/1410.2597),  [arxiv.org/1507.06739](http://arxiv.org/abs/1507.06739v1)




# Additive noise

<img src="http://statweb.stanford.edu/~jtaylo/talks/bids2015/ROC_3.png" width="500">

Can do even better than data carving!